In [75]:
import pandas as pd
import numpy as np

import bamboolib as bam

In [76]:
URL = 'https://raw.githubusercontent.com/globaldothealth/monkeypox/main/latest.csv'

data = pd.read_csv(URL)

data

,ID,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,...,Travel_history_country,Genomics_Metadata,Confirmation_method,Source,Source_II,Date_entry,Date_last_modified,Source_III,Source_IV,Country_ISO3
0,1,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,2022-04-29,2022-05-06,rash,...,Nigeria,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,https://www.who.int/emergencies/disease-outbre...,2022-05-18,2022-05-18,NaN,NaN,GBR
1,2,confirmed,Guy's and St Thomas Hospital London,London,England,NaN,NaN,2022-05-05,2022-05-12,rash,...,NaN,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
2,3,confirmed,London,London,England,NaN,NaN,2022-04-30,2022-05-13,vesicular rash,...,NaN,West African Clade,RT-PCR,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
3,4,confirmed,London,London,England,NaN,male,NaN,2022-05-15,vesicular rash,...,NaN,West African Clade,NaN,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
4,5,confirmed,London,London,England,NaN,male,NaN,2022-05-15,vesicular rash,...,NaN,West African Clade,NaN,https://www.gov.uk/government/news/monkeypox-c...,NaN,2022-05-18,2022-05-18,NaN,NaN,GBR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3067,3068,confirmed,NaN,NaN,Switzerland,NaN,NaN,NaN,2022-06-20,NaN,...,NaN,NaN,NaN,https://www.bag.admin.ch/bag/en/home/krankheit...,NaN,2022-06-20,2022-06-20,NaN,NaN,CHE
3068,3069,confirmed,NaN,NaN,Switzerland,NaN,NaN,NaN,2022-06-20,NaN,...,NaN,NaN,NaN,https://www.bag.admin.ch/bag/en/home/krankheit...,NaN,2022-06-20,2022-06-20,NaN,NaN,CHE
3069,3070,confirmed,NaN,NaN,Switzerland,NaN,NaN,NaN,2022-06-20,NaN,...,NaN,NaN,NaN,https://www.bag.admin.ch/bag/en/home/krankheit...,NaN,2022-06-20,2022-06-20,NaN,NaN,CHE
3070,3071,suspected,Missouri,Kansas City,United States,NaN,NaN,NaN,NaN,NaN,...,United States,NaN,NaN,https://www.ntd.com/first-probable-monkeypox-c...,NaN,2022-06-20,2022-06-20,NaN,NaN,USA


In [77]:
data_pox =data[data.columns[data.isna().sum()/data.shape[0]<0.85]]
data_pox = data_pox.loc[~(data_pox['Status'].isin(['discarded']))]
data_pox = data_pox.drop(columns=['Location', 'City', 'Gender', 'Source_II', 'ID', 'Source','Date_last_modified'])

suspected_data_pox = data_pox.loc[data_pox['Status'].isin(['suspected'])]
suspected_data_pox['date'] = pd.to_datetime(suspected_data_pox['Date_entry'], format='%Y-%m-%d')
confirmed_data_pox = suspected_data_pox.groupby(['Country', 'date']).agg(suspected=('Status', 'size')).reset_index()


confirmed_data_pox = data_pox.loc[data_pox['Status'].isin(['confirmed'])]
confirmed_data_pox['date'] = pd.to_datetime(confirmed_data_pox['Date_confirmation'], format='%Y-%m-%d')
confirmed_data_pox = confirmed_data_pox.groupby(['Country', 'date']).agg(confirmed=('Status', 'size')).reset_index()
#confirmed_data_pox['confirmed'] = confirmed_data_pox.groupby(['Country'])['daily_cases'].cumsum()
#confirmed_data_pox = confirmed_data_pox.sort_values(by=['date'], ascending = True)

In [78]:
confirmed_data_pox.Country.unique()

array(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada',
       'Chile', 'Czech Republic', 'Denmark', 'England', 'Finland',
       'France', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece',
       'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Latvia',
       'Lebanon', 'Luxembourg', 'Malta', 'Mexico', 'Morocco',
       'Netherlands', 'Northern Ireland', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Scotland', 'Serbia', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'United Arab Emirates', 'United States',
       'Venezuela', 'Wales'], dtype=object)

In [79]:
len(confirmed_data_pox.Country.unique())

44

In [80]:
print(confirmed_data_pox.Country.unique())

['Argentina' 'Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile'
 'Czech Republic' 'Denmark' 'England' 'Finland' 'France' 'Georgia'
 'Germany' 'Ghana' 'Gibraltar' 'Greece' 'Hungary' 'Iceland' 'Ireland'
 'Israel' 'Italy' 'Latvia' 'Lebanon' 'Luxembourg' 'Malta' 'Mexico'
 'Morocco' 'Netherlands' 'Northern Ireland' 'Norway' 'Poland' 'Portugal'
 'Romania' 'Scotland' 'Serbia' 'Slovenia' 'Spain' 'Sweden' 'Switzerland'
 'United Arab Emirates' 'United States' 'Venezuela' 'Wales']


In [81]:
from datetime import date


def map_dateRange(df, w_countries):
    date_rng = pd.date_range(start='5/1/2022', end=confirmed_data_pox['date'].max())
    df_date = pd.DataFrame(date_rng,  columns=['date'])
    df = df[df['Country']==w_countries]
    df = df.merge(df_date, how='outer', on = ['date'])
    df['Country'] = df['Country'].fillna(w_countries)
    df['confirmed']=df['confirmed'].fillna(0)
    df = df.sort_values(by=['date'],  ascending=True, axis=0)
    return df

In [82]:
argentina = map_dateRange(confirmed_data_pox,'Argentina')
australia = map_dateRange(confirmed_data_pox,'Australia')
austria = map_dateRange(confirmed_data_pox,'Austria')
belgium = map_dateRange(confirmed_data_pox,'Belgium')
brazil = map_dateRange(confirmed_data_pox,'Brazil')
canada = map_dateRange(confirmed_data_pox,'Canada')
czechrepublic = map_dateRange(confirmed_data_pox,'Czech Republic')
Chile =  map_dateRange(confirmed_data_pox, 'Chile')
Denmark = map_dateRange(confirmed_data_pox,'Denmark')
England = map_dateRange(confirmed_data_pox,'England')
Finland = map_dateRange(confirmed_data_pox,'Finland')
France = map_dateRange(confirmed_data_pox,'France')
Germany = map_dateRange(confirmed_data_pox,'Germany')
Georgia = map_dateRange(confirmed_data_pox,'Georgia')
Ghana = map_dateRange(confirmed_data_pox,'Ghana')
Gibraltar = map_dateRange(confirmed_data_pox,'Gibraltar')
Greece = map_dateRange(confirmed_data_pox,'Greece')
Hungary = map_dateRange(confirmed_data_pox,'Hungary')
Iceland = map_dateRange(confirmed_data_pox,'Iceland')
Ireland = map_dateRange(confirmed_data_pox,'Ireland')
Israel = map_dateRange(confirmed_data_pox,'Israel')
Italy = map_dateRange(confirmed_data_pox,'Italy')
Latvia = map_dateRange(confirmed_data_pox,'Latvia')
Lebanon = map_dateRange(confirmed_data_pox,'Lebanon')
Luxembourg = map_dateRange(confirmed_data_pox,'Luxembourg')
Mexico = map_dateRange(confirmed_data_pox,'Mexico')
Malta = map_dateRange(confirmed_data_pox,'Malta')
Morocco = map_dateRange(confirmed_data_pox,'Morocco')
Netherlands = map_dateRange(confirmed_data_pox,'Netherlands')
NorthernIreland = map_dateRange(confirmed_data_pox,'Northern Ireland')
Norway = map_dateRange(confirmed_data_pox,'Norway')
Poland = map_dateRange(confirmed_data_pox,'Poland')
Portugal = map_dateRange(confirmed_data_pox,'Portugal')
Romania = map_dateRange(confirmed_data_pox,'Romania')
Scotland = map_dateRange(confirmed_data_pox,'Scotland')
Serbia = map_dateRange(confirmed_data_pox,'Serbia')
Slovenia = map_dateRange(confirmed_data_pox,'Slovenia')
Spain = map_dateRange(confirmed_data_pox,'Spain')
Sweden = map_dateRange(confirmed_data_pox,'Sweden')
Switzerland = map_dateRange(confirmed_data_pox,'Switzerland')
UnitedArabEmirates = map_dateRange(confirmed_data_pox,'United Arab Emirates')
UnitedStates = map_dateRange(confirmed_data_pox,'United States')
Wales = map_dateRange(confirmed_data_pox,'Wales')







In [83]:
df_monkeypox = pd.concat([argentina, australia, austria, 
                          belgium, brazil, canada, czechrepublic, 
                          Denmark, England, Finland, France, Germany, 
                          Ghana, Gibraltar, Greece, Hungary, Ireland, Israel, 
                          Italy, Latvia, Mexico, Malta, Morocco, Netherlands, 
                          NorthernIreland, Norway, Poland, Portugal, Scotland, Slovenia, 
                          Spain, Sweden, Switzerland, UnitedArabEmirates, 
                          UnitedStates, Wales, Georgia, Chile, Iceland, Lebanon, Luxembourg, Romania, Serbia], axis=0, ignore_index=True)


In [84]:
df_monkeypox['acc_confirmed'] = df_monkeypox.groupby(['Country'])['confirmed'].cumsum()
df_monkeypox['acc_confirmed'] = pd.to_numeric(df_monkeypox['acc_confirmed'], downcast='integer', errors='coerce')
df_monkeypox['confirmed'] = pd.to_numeric(df_monkeypox['confirmed'], downcast='integer', errors='coerce')


In [85]:
df_world = df_monkeypox.groupby(['date']).agg(acc_confirmed_sum=('acc_confirmed', 'sum'), confirmed_sum=('confirmed', 'sum')).reset_index()
df_world['World'] = 'World'

In [86]:
df_world = df_world[['World'] + ['date', 'acc_confirmed_sum', 'confirmed_sum']]
df_world['confirmed_sum'] = pd.to_numeric(df_world['confirmed_sum'], downcast='integer', errors='coerce')
df_world = df_world[['World', 'date', 'confirmed_sum'] + ['acc_confirmed_sum']]
df_world = df_world.rename(columns={'confirmed_sum': 'confirmed', 'acc_confirmed_sum': 'acc_confirmed', 'World': 'Country'})
df_monkeypox = pd.concat([df_world, df_monkeypox], axis=0, ignore_index=True)
df_monkeypox

,Country,date,confirmed,acc_confirmed
0,World,2022-05-01,0,0
1,World,2022-05-02,0,0
2,World,2022-05-03,0,0
3,World,2022-05-04,0,0
4,World,2022-05-05,0,0
...,...,...,...,...
2239,Serbia,2022-06-16,0,0
2240,Serbia,2022-06-17,1,1
2241,Serbia,2022-06-18,0,1
2242,Serbia,2022-06-19,0,1


In [87]:
#get world count
world_count = df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-1]
world_count

2676

In [88]:
#set update date

df_monkeypox['date'].iloc[-1]

Timestamp('2022-06-20 00:00:00')

In [89]:
df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-1]-df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]['acc_confirmed'].iloc[-2]



90

In [90]:
geo = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')


In [91]:
df_monkeypox_uk = df_monkeypox.loc[df_monkeypox['Country'].isin(['England', 'Northern Ireland', 'Scotland', 'Wales'])]


In [92]:
df_monkeypox_word = df_monkeypox.loc[df_monkeypox['Country'].isin(['World'])]


In [93]:
df_monkeypox = df_monkeypox.loc[~(df_monkeypox['Country'].isin(['World', 'England', 'Northern Ireland', 'Scotland', 'Wales']))]


In [94]:
df_monkeypox_uk['latitude'] = 55.378051
df_monkeypox_uk['longitude'] = -3.435973

In [95]:
df_monkeypox = pd.merge(df_monkeypox, geo[['country', 'latitude', 'longitude']], how='inner', left_on=['Country'], right_on=['country'])
df_monkeypox = df_monkeypox.reset_index()


In [96]:
df_monkeypox = pd.concat([df_monkeypox, df_monkeypox_uk], axis=0, ignore_index=True)
df_monkeypox = df_monkeypox.drop(columns=['index', 'country'])


In [97]:
df_monkeypox_word['latitude']=0
df_monkeypox_word['longitude']=0

In [98]:
df_monkeypox = pd.concat([df_monkeypox, df_monkeypox_word], axis=0, ignore_index=True)

In [99]:
df_monkeypox.to_csv('df_monkeyPox.csv', index_label=False)

In [107]:
#df_monkeypox[df_monkeypox['Country']=='World']['acc_confirmed'].max()

2676

In [104]:
df_monkeypox
print(df_monkeypox.Country.unique())

['Argentina' 'Australia' 'Austria' 'Belgium' 'Brazil' 'Canada'
 'Czech Republic' 'Denmark' 'Finland' 'France' 'Germany' 'Ghana'
 'Gibraltar' 'Greece' 'Hungary' 'Ireland' 'Israel' 'Italy' 'Latvia'
 'Mexico' 'Malta' 'Morocco' 'Netherlands' 'Norway' 'Poland' 'Portugal'
 'Slovenia' 'Spain' 'Sweden' 'Switzerland' 'United Arab Emirates'
 'United States' 'Georgia' 'Chile' 'Iceland' 'Lebanon' 'Luxembourg'
 'Romania' 'Serbia' 'England' 'Northern Ireland' 'Scotland' 'Wales'
 'World']


In [ ]:
f"{world_count:,.0f}"

In [101]:
#set data for map

df_monkeypox.groupby(['Country', 'latitude', 'longitude'])['acc_confirmed'].max().reset_index()

,Country,latitude,longitude,acc_confirmed
0,Argentina,-38.416097,-63.616672,5
1,Australia,-25.274398,133.775136,13
2,Austria,47.516231,14.550072,11
3,Belgium,50.503887,4.469936,62
4,Brazil,-14.235004,-51.925280,7
5,Canada,56.130366,-106.346771,177
6,Chile,-35.675147,-71.542969,1
7,Czech Republic,49.817492,15.472962,6
8,Denmark,56.263920,9.501785,8
9,England,55.378051,-3.435973,550


In [102]:
# Create dictionary of list
testlist = df_monkeypox[['Country', 'latitude', 'longitude']]
dict_of_locations = testlist.set_index('Country')[['latitude', 'longitude']].T.to_dict('dict')

In [103]:
dict_of_locations

{'Argentina': {'latitude': -38.416097, 'longitude': -63.616672},
 'Australia': {'latitude': -25.274398, 'longitude': 133.775136},
 'Austria': {'latitude': 47.516231, 'longitude': 14.550072},
 'Belgium': {'latitude': 50.503887, 'longitude': 4.469936},
 'Brazil': {'latitude': -14.235004, 'longitude': -51.92528},
 'Canada': {'latitude': 56.130366, 'longitude': -106.346771},
 'Czech Republic': {'latitude': 49.817492, 'longitude': 15.472962},
 'Denmark': {'latitude': 56.26392, 'longitude': 9.501785},
 'Finland': {'latitude': 61.92411, 'longitude': 25.748151},
 'France': {'latitude': 46.227638, 'longitude': 2.213749},
 'Germany': {'latitude': 51.165691, 'longitude': 10.451526},
 'Ghana': {'latitude': 7.946527, 'longitude': -1.023194},
 'Gibraltar': {'latitude': 36.137741, 'longitude': -5.345374},
 'Greece': {'latitude': 39.074208, 'longitude': 21.824312},
 'Hungary': {'latitude': 47.162494, 'longitude': 19.503304},
 'Ireland': {'latitude': 53.41291, 'longitude': -8.24389},
 'Israel': {'latit